# ＃生成交易帐号：

<ol>
<li>生成256位（64字节）私钥</li>
<li>非对称算法生成公钥</li>
<li>计算公钥的SHA-256</li>
<li>计算ripemd-160</li>
<li>在头上加上地址版本</li>
<li>计算SHA-256</li>
<li>计算SHA-256</li>
<li>取第7步的前4个字节，也就是8个16进制个位数，也就是字符串前8位</li>
<li>把第8步的4个字节内容添加到第5步结果后面作为校验</li>
<li>对第9步的结果做base58编码，得到最终地址</li>
</ol>

In [2]:
import base58#2进制buff变成字符串
import binascii as ba
import ecdsa　#椭圆曲线的非对称密钥生成
import hashlib
import random

In [9]:
#生成256位（64字节）私钥
def create_priv_key():#256bit = 64*4bit(0-F)
    digits = ['%x' % random.randrange(16) for i in range(64)]
    return ''.join(digits)

In [10]:
priv_key = create_priv_key()

In [11]:
priv_key

'7ed06dfcb0705ac89480005a57a93de447cfbac8752ec105e3b8cadb7bc384c0'

In [17]:
#非对称算法生成公钥
def build_pub_key(priv_key):
    buff = bytes.fromhex(priv_key)#转为二进制buff
    print(type(buff))
    print(buff)
    sk= ecdsa.SigningKey.from_string(buff,curve=ecdsa.SECP256k1)
    pub_key = b'\x04' + sk.verifying_key.to_string()
    print(pub_key)
    return ba.b2a_hex(pub_key).decode()#

In [18]:
pub_key = build_pub_key(priv_key)

<class 'bytes'>
b'~\xd0m\xfc\xb0pZ\xc8\x94\x80\x00ZW\xa9=\xe4G\xcf\xba\xc8u.\xc1\x05\xe3\xb8\xca\xdb{\xc3\x84\xc0'
b'\x049(\x1c\x99\x9e\xb9\x14{\x15Lc\xc9\xcc\x08\xbf\x06U\x0e\xf5\xba\xa6F5\xee[fQ\x1e\xbe\x11\xf7\x80\x949t\xa9\xf3\x98\xceQhtv|%ce\xf0;\xcc\x13F\x9f\x0b\xcbUI>\xf12#\x93\xa8<'


In [16]:
pub_key

'0439281c999eb9147b154c63c9cc08bf06550ef5baa64635ee5b66511ebe11f780943974a9f398ce516874767c256365f03bcc13469f0bcb55493ef1322393a83c'

In [19]:
#计算公钥的SHA-256
def sha256(key):
    sha = hashlib.sha256(bytes.fromhex(key))
    return ba.hexlify(sha.digest()).decode()

In [24]:
def ripemd160(key):
    rm = hashlib.new('ripemd160')
    rm.update(bytes.fromhex(key))
    return ba.hexlify(rm.digest()).decode()

In [25]:
r3 = sha256(pub_key)
print(r3)

82bac5e3c46d8d0d71926aad3d4df2504dce0986c790d9284b220db53e6c03d1


In [26]:
r4 = ripemd160(r3)
r4

'88d0dad982cadfe60de4e2edd4817bca6e02e487'

In [27]:
r5 = '00'+r4;

In [28]:
r6 = sha256(r5)
r7 = sha256(r6)
r7

'a341043ed981cf4d32737345f3aba3cc78d5393a17596e3faa53d2c044fb4d65'

In [29]:
r8 = r7[:8]
r8

'a341043e'

In [30]:
r9 = r5+r8
r9

'0088d0dad982cadfe60de4e2edd4817bca6e02e487a341043e'

In [31]:
addr = base58.b58encode(bytes.fromhex(r9))

In [32]:
addr

'1DUR6afpHosVRcuQgxGk476PQva2M35vyT'